In [1]:
# Импортирование библиотек для иницализации основных параметров системы
from fortran_calc_tools import Fazy, System, Garmoniki, Rachety, Looper, AttribHolder, PodStans, PhaseHolder

# Импортирование неодходимых библиотек
from os import path 
import time
import pandas as pd, numpy as np
from collections import namedtuple
from calc_tools import pairwise_1, pairwise_2
from scipy.linalg import logm, expm, sqrtm
from multiprocessing import Pool
from numpy.linalg import inv
from numpy import linalg as LA

# Импортирование библиотек для визуализации данных !!!!!
import matplotlib.pyplot as plot
import matplotlib as mpl
import seaborn as sns
import plotly as pt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

In [2]:
# Задание значении характеристик материала для фаз и троса
Faza_A = Fazy("Фаза А", 35.336, 1.0, 150.0, 0.0, 19.0)
Faza_B = Fazy("Фаза В", 35.336, 1.0, 150.0, 6.3, 19.0)
Faza_C = Fazy("Фаза С", 35.336, 1.0, 150.0, 4.2, 25.0)
Tross = Fazy("Тросс", 17.336, 4000.0, 50.0, 2.1, 28.0)

# В данном случае у нас будет 50 гармоник от [1 до 50]
garmoniki = Garmoniki(50)

# Создание объекта для проведения расчетов
rashet = Rachety()

# Инициализация объекта для подстанции "Зеленая" и чтение файла
basepath = path.abspath('')
filepath = path.abspath(path.join(basepath, "..", path.join("data", "Данные ПС Сарепта1.xls")))
PodS_zelenaya = PodStans(filepath, volt=110, harm_num=49)

dummy_str_input = "fuzzBuzz for testing"
dummy_int_input = 404

system = System(PodS_zelenaya, garmoniki, rashet, Fazy, dummy_str_input, dummy_str_input, dummy_int_input, dummy_int_input,
     dummy_int_input, dummy_int_input, dummy_int_input)

# Установка конфигурации для выведения данных
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
#                   !!! DEBUGGING PROCESS HERE !!!

In [30]:
UM, FUM, AIM, FIM, funu, fu, funi, fi, knsu, knsi, rmsu, rmsi = system.line_matrix(1)

UM1 = np.zeros((700, UM.shape[1],UM.shape[2]), UM.dtype)
UM2 = np.zeros((700, UM.shape[1],UM.shape[2]), UM.dtype)
AIM1 = np.zeros((700, UM.shape[1],UM.shape[2]), UM.dtype)
AIM2 = np.zeros((700, UM.shape[1],UM.shape[2]), UM.dtype)
PPR1 = np.zeros((700), UM.dtype)

UK1 = [complex(0, 0) for _ in range(8)]
AIK1 = [complex(0, 0) for _ in range(8)]

UK10 = complex(0, 0); UK11 = complex(0, 0); UK12 = complex(0, 0);
AIK10 = complex(0, 0); AIK11 = complex(0, 0); AIK12 = complex(0, 0);
AL = complex(0, 0);
PP1 = 0.0

In [31]:
# 11 - 13
for i in range(560+1):
    for p in range(3):
        UM[i,0,p] = funu[i,p]*np.sqrt(2)
        FUM[i,0,p]=fu[i,p]*np.pi/180
        UM1[i,0,p]=UM[i,0,p]*np.cos(FUM[i,0,p])
        UM2[i,0,p]=UM[i,0,p]*np.sin(FUM[i,0,p])
        
# 13 - 14
for i in range(560+1):
    for p in range(3):
        for k in range(49):
            UM[i,k,p]=UM[i,k,p]*UM[i,1,p]/100.
            FUM[i,k,p]=FUM[i,k,p]*np.pi/180.
            UM1[i,k,p]=UM[i,k,p]*np.cos(FUM[i,k,p])
            UM2[i,k,p]=UM[i,k,p]*np.sin(FUM[i,k,p])

# 14 - 16
for i in range(560+1):
    for p in range(3):
        AIM[i,0,p]=funi[i,p]*np.sqrt(2)
        FIM[i,0,p]=fi[i,p]*np.pi/180
        AIM1[i,0,p]=AIM[i,0,p]*np.cos(FIM[i,0,p])
        AIM2[i,0,p]=AIM[i,0,p]*np.sin(FIM[i,0,p])
        
# 16 - 117
for i in range(560+1):
    for p in range(3):
        for k in range(49):
            AIM[i,k,p]=AIM[i,k,p]*AIM[i,0,p]/100
            FIM[i,k,p]=FIM[i,k,p]*np.pi/180
            AIM1[i,k,p]=AIM[i,k,p]*np.cos(FIM[i,k,p])
            AIM2[i,k,p]=AIM[i,k,p]*np.sin(FIM[i,k,p])

In [32]:
# 117 - 1700 - 1500
MM, MPR, MTR, DT, MT = 5, 3, 1, 2.5, 5
M = MPR + MTR
MMT = MM / MT

if M <= 6:
    M1 = M
if M > 6:
    M1 = 6

M10 = 2 * M
M20 = 4 * M

n = 0
k = 0

next_outer = False
update_inner = True

inner_loop = 49 # k or K3
outer_loop = 560 # n or  N3
PR = 0

while True:
    PR += 1
    
    if n == outer_loop:
        break
#     print("Outer,  Inner, PR: ", n, k, PR, update_inner)
    print("RUN BLOCK 1", k, n, PR)
    UK1[0] = complex(UM1[n,k,0], UM2[n,k,0])
    UK1[1] = complex(UM1[n,k,1], UM2[n,k,1])
    UK1[2] = complex(UM1[n,k,2], UM2[n,k,2])
    UK1[3] = complex(0, 0)
    AIK1[0] = complex(AIM1[n,k,0], AIM2[n,k,0])
    AIK1[1] = complex(AIM1[n,k,1], AIM2[n,k,1])
    AIK1[2] = complex(AIM1[n,k,2], AIM2[n,k,2])
    AIK1[3] = complex(0, 0)

    if not (k > 0) or (k == 0 and PR ==2): # ---> 1111
        print("RUN BLOCK 2", k, n, PR)
        UK10=(UK1[0]+UK1[1]+UK1[2])/3
        UK11=(UK1[0]+UK1[1]*AL+UK1[2]*AL**2)/3
        UK12=(UK1[0]+UK1[1]*AL**2+UK1[2]*AL)/3
        SKU2=np.sqrt(UK12.real**2+UK12.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
        SKU0=np.sqrt(UK10.real**2+UK10.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
        UK1[0]=UK11
        UK1[1]=UK11*AL**2
        UK1[2]=UK11*AL
        AIK10=(AIK1[0]+AIK1[1]+AIK1[2])/3
        AIK11=(AIK1[0]+AIK1[1]*AL+AIK1[2]*AL**2)/3
        AIK12=(AIK1[0]+AIK1[1]*AL**2+AIK1[2]*AL)/3
        SKI2=np.sqrt(AIK12.real**2+AIK12.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
        SKI0=np.sqrt(AIK10.real**2+AIK10.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
        AIK1[1]=AIK11
        AIK1[2]=AIK11*AL**2
        AIK1[3]=AIK11*AL
    
    print("RUN BLOCK 3", k, n, PR)
#     raschet(UK1,AIK1,k,n,PPP,PP1,PP2,ppp1,ppp2,ppp3,ppp4,ppp5,ppp6,ppp7,ppp8)

    if update_inner:
        if k < inner_loop-1:
            k += 1
            next_outer = False
        else:
            k = 0
            next_outer = True

    if next_outer:
        if n < outer_loop:
            n += 1
        else:
            n = 0

    if k == 0 and PR == 1:
        update_inner = False  # ---> 1700
        continue
    if PR == 2:
        update_inner = True  # ---> 1500
        PR = 0
        continue

RUN BLOCK 1 0 0 1
RUN BLOCK 2 0 0 1
RUN BLOCK 3 0 0 1
RUN BLOCK 1 1 0 2
RUN BLOCK 3 1 0 2
RUN BLOCK 1 2 0 1
RUN BLOCK 3 2 0 1
RUN BLOCK 1 3 0 2
RUN BLOCK 3 3 0 2
RUN BLOCK 1 4 0 1
RUN BLOCK 3 4 0 1
RUN BLOCK 1 5 0 2
RUN BLOCK 3 5 0 2
RUN BLOCK 1 6 0 1
RUN BLOCK 3 6 0 1
RUN BLOCK 1 7 0 2
RUN BLOCK 3 7 0 2
RUN BLOCK 1 8 0 1
RUN BLOCK 3 8 0 1
RUN BLOCK 1 9 0 2
RUN BLOCK 3 9 0 2
RUN BLOCK 1 10 0 1
RUN BLOCK 3 10 0 1
RUN BLOCK 1 11 0 2
RUN BLOCK 3 11 0 2
RUN BLOCK 1 12 0 1
RUN BLOCK 3 12 0 1
RUN BLOCK 1 13 0 2
RUN BLOCK 3 13 0 2
RUN BLOCK 1 14 0 1
RUN BLOCK 3 14 0 1
RUN BLOCK 1 15 0 2
RUN BLOCK 3 15 0 2
RUN BLOCK 1 16 0 1
RUN BLOCK 3 16 0 1
RUN BLOCK 1 17 0 2
RUN BLOCK 3 17 0 2
RUN BLOCK 1 18 0 1
RUN BLOCK 3 18 0 1
RUN BLOCK 1 19 0 2
RUN BLOCK 3 19 0 2
RUN BLOCK 1 20 0 1
RUN BLOCK 3 20 0 1
RUN BLOCK 1 21 0 2
RUN BLOCK 3 21 0 2
RUN BLOCK 1 22 0 1
RUN BLOCK 3 22 0 1
RUN BLOCK 1 23 0 2
RUN BLOCK 3 23 0 2
RUN BLOCK 1 24 0 1
RUN BLOCK 3 24 0 1
RUN BLOCK 1 25 0 2
RUN BLOCK 3 25 0 2
RUN BLOCK 1 26

RUN BLOCK 3 25 16 2
RUN BLOCK 1 26 16 1
RUN BLOCK 3 26 16 1
RUN BLOCK 1 27 16 2
RUN BLOCK 3 27 16 2
RUN BLOCK 1 28 16 1
RUN BLOCK 3 28 16 1
RUN BLOCK 1 29 16 2
RUN BLOCK 3 29 16 2
RUN BLOCK 1 30 16 1
RUN BLOCK 3 30 16 1
RUN BLOCK 1 31 16 2
RUN BLOCK 3 31 16 2
RUN BLOCK 1 32 16 1
RUN BLOCK 3 32 16 1
RUN BLOCK 1 33 16 2
RUN BLOCK 3 33 16 2
RUN BLOCK 1 34 16 1
RUN BLOCK 3 34 16 1
RUN BLOCK 1 35 16 2
RUN BLOCK 3 35 16 2
RUN BLOCK 1 36 16 1
RUN BLOCK 3 36 16 1
RUN BLOCK 1 37 16 2
RUN BLOCK 3 37 16 2
RUN BLOCK 1 38 16 1
RUN BLOCK 3 38 16 1
RUN BLOCK 1 39 16 2
RUN BLOCK 3 39 16 2
RUN BLOCK 1 40 16 1
RUN BLOCK 3 40 16 1
RUN BLOCK 1 41 16 2
RUN BLOCK 3 41 16 2
RUN BLOCK 1 42 16 1
RUN BLOCK 3 42 16 1
RUN BLOCK 1 43 16 2
RUN BLOCK 3 43 16 2
RUN BLOCK 1 44 16 1
RUN BLOCK 3 44 16 1
RUN BLOCK 1 45 16 2
RUN BLOCK 3 45 16 2
RUN BLOCK 1 46 16 1
RUN BLOCK 3 46 16 1
RUN BLOCK 1 47 16 2
RUN BLOCK 3 47 16 2
RUN BLOCK 1 48 16 1
RUN BLOCK 3 48 16 1
RUN BLOCK 1 0 17 2
RUN BLOCK 2 0 17 2
RUN BLOCK 3 0 17 2
RUN

RUN BLOCK 1 20 30 1
RUN BLOCK 3 20 30 1
RUN BLOCK 1 21 30 2
RUN BLOCK 3 21 30 2
RUN BLOCK 1 22 30 1
RUN BLOCK 3 22 30 1
RUN BLOCK 1 23 30 2
RUN BLOCK 3 23 30 2
RUN BLOCK 1 24 30 1
RUN BLOCK 3 24 30 1
RUN BLOCK 1 25 30 2
RUN BLOCK 3 25 30 2
RUN BLOCK 1 26 30 1
RUN BLOCK 3 26 30 1
RUN BLOCK 1 27 30 2
RUN BLOCK 3 27 30 2
RUN BLOCK 1 28 30 1
RUN BLOCK 3 28 30 1
RUN BLOCK 1 29 30 2
RUN BLOCK 3 29 30 2
RUN BLOCK 1 30 30 1
RUN BLOCK 3 30 30 1
RUN BLOCK 1 31 30 2
RUN BLOCK 3 31 30 2
RUN BLOCK 1 32 30 1
RUN BLOCK 3 32 30 1
RUN BLOCK 1 33 30 2
RUN BLOCK 3 33 30 2
RUN BLOCK 1 34 30 1
RUN BLOCK 3 34 30 1
RUN BLOCK 1 35 30 2
RUN BLOCK 3 35 30 2
RUN BLOCK 1 36 30 1
RUN BLOCK 3 36 30 1
RUN BLOCK 1 37 30 2
RUN BLOCK 3 37 30 2
RUN BLOCK 1 38 30 1
RUN BLOCK 3 38 30 1
RUN BLOCK 1 39 30 2
RUN BLOCK 3 39 30 2
RUN BLOCK 1 40 30 1
RUN BLOCK 3 40 30 1
RUN BLOCK 1 41 30 2
RUN BLOCK 3 41 30 2
RUN BLOCK 1 42 30 1
RUN BLOCK 3 42 30 1
RUN BLOCK 1 43 30 2
RUN BLOCK 3 43 30 2
RUN BLOCK 1 44 30 1
RUN BLOCK 3 44 30 1


RUN BLOCK 1 20 46 1
RUN BLOCK 3 20 46 1
RUN BLOCK 1 21 46 2
RUN BLOCK 3 21 46 2
RUN BLOCK 1 22 46 1
RUN BLOCK 3 22 46 1
RUN BLOCK 1 23 46 2
RUN BLOCK 3 23 46 2
RUN BLOCK 1 24 46 1
RUN BLOCK 3 24 46 1
RUN BLOCK 1 25 46 2
RUN BLOCK 3 25 46 2
RUN BLOCK 1 26 46 1
RUN BLOCK 3 26 46 1
RUN BLOCK 1 27 46 2
RUN BLOCK 3 27 46 2
RUN BLOCK 1 28 46 1
RUN BLOCK 3 28 46 1
RUN BLOCK 1 29 46 2
RUN BLOCK 3 29 46 2
RUN BLOCK 1 30 46 1
RUN BLOCK 3 30 46 1
RUN BLOCK 1 31 46 2
RUN BLOCK 3 31 46 2
RUN BLOCK 1 32 46 1
RUN BLOCK 3 32 46 1
RUN BLOCK 1 33 46 2
RUN BLOCK 3 33 46 2
RUN BLOCK 1 34 46 1
RUN BLOCK 3 34 46 1
RUN BLOCK 1 35 46 2
RUN BLOCK 3 35 46 2
RUN BLOCK 1 36 46 1
RUN BLOCK 3 36 46 1
RUN BLOCK 1 37 46 2
RUN BLOCK 3 37 46 2
RUN BLOCK 1 38 46 1
RUN BLOCK 3 38 46 1
RUN BLOCK 1 39 46 2
RUN BLOCK 3 39 46 2
RUN BLOCK 1 40 46 1
RUN BLOCK 3 40 46 1
RUN BLOCK 1 41 46 2
RUN BLOCK 3 41 46 2
RUN BLOCK 1 42 46 1
RUN BLOCK 3 42 46 1
RUN BLOCK 1 43 46 2
RUN BLOCK 3 43 46 2
RUN BLOCK 1 44 46 1
RUN BLOCK 3 44 46 1


RUN BLOCK 2 0 63 2
RUN BLOCK 3 0 63 2
RUN BLOCK 1 0 64 1
RUN BLOCK 2 0 64 1
RUN BLOCK 3 0 64 1
RUN BLOCK 1 1 64 2
RUN BLOCK 3 1 64 2
RUN BLOCK 1 2 64 1
RUN BLOCK 3 2 64 1
RUN BLOCK 1 3 64 2
RUN BLOCK 3 3 64 2
RUN BLOCK 1 4 64 1
RUN BLOCK 3 4 64 1
RUN BLOCK 1 5 64 2
RUN BLOCK 3 5 64 2
RUN BLOCK 1 6 64 1
RUN BLOCK 3 6 64 1
RUN BLOCK 1 7 64 2
RUN BLOCK 3 7 64 2
RUN BLOCK 1 8 64 1
RUN BLOCK 3 8 64 1
RUN BLOCK 1 9 64 2
RUN BLOCK 3 9 64 2
RUN BLOCK 1 10 64 1
RUN BLOCK 3 10 64 1
RUN BLOCK 1 11 64 2
RUN BLOCK 3 11 64 2
RUN BLOCK 1 12 64 1
RUN BLOCK 3 12 64 1
RUN BLOCK 1 13 64 2
RUN BLOCK 3 13 64 2
RUN BLOCK 1 14 64 1
RUN BLOCK 3 14 64 1
RUN BLOCK 1 15 64 2
RUN BLOCK 3 15 64 2
RUN BLOCK 1 16 64 1
RUN BLOCK 3 16 64 1
RUN BLOCK 1 17 64 2
RUN BLOCK 3 17 64 2
RUN BLOCK 1 18 64 1
RUN BLOCK 3 18 64 1
RUN BLOCK 1 19 64 2
RUN BLOCK 3 19 64 2
RUN BLOCK 1 20 64 1
RUN BLOCK 3 20 64 1
RUN BLOCK 1 21 64 2
RUN BLOCK 3 21 64 2
RUN BLOCK 1 22 64 1
RUN BLOCK 3 22 64 1
RUN BLOCK 1 23 64 2
RUN BLOCK 3 23 64 2
RUN

...

RUN BLOCK 1 42 474 1
RUN BLOCK 3 42 474 1
RUN BLOCK 1 43 474 2
RUN BLOCK 3 43 474 2
RUN BLOCK 1 44 474 1
RUN BLOCK 3 44 474 1
RUN BLOCK 1 45 474 2
RUN BLOCK 3 45 474 2
RUN BLOCK 1 46 474 1
RUN BLOCK 3 46 474 1
RUN BLOCK 1 47 474 2
RUN BLOCK 3 47 474 2
RUN BLOCK 1 48 474 1
RUN BLOCK 3 48 474 1
RUN BLOCK 1 0 475 2
RUN BLOCK 2 0 475 2
RUN BLOCK 3 0 475 2
RUN BLOCK 1 0 476 1
RUN BLOCK 2 0 476 1
RUN BLOCK 3 0 476 1
RUN BLOCK 1 1 476 2
RUN BLOCK 3 1 476 2
RUN BLOCK 1 2 476 1
RUN BLOCK 3 2 476 1
RUN BLOCK 1 3 476 2
RUN BLOCK 3 3 476 2
RUN BLOCK 1 4 476 1
RUN BLOCK 3 4 476 1
RUN BLOCK 1 5 476 2
RUN BLOCK 3 5 476 2
RUN BLOCK 1 6 476 1
RUN BLOCK 3 6 476 1
RUN BLOCK 1 7 476 2
RUN BLOCK 3 7 476 2
RUN BLOCK 1 8 476 1
RUN BLOCK 3 8 476 1
RUN BLOCK 1 9 476 2
RUN BLOCK 3 9 476 2
RUN BLOCK 1 10 476 1
RUN BLOCK 3 10 476 1
RUN BLOCK 1 11 476 2
RUN BLOCK 3 11 476 2
RUN BLOCK 1 12 476 1
RUN BLOCK 3 12 476 1
RUN BLOCK 1 13 476 2
RUN BLOCK 3 13 476 2
RUN BLOCK 1 14 476 1
RUN BLOCK 3 14 476 1
RUN BLOCK 1 15 4

RUN BLOCK 3 20 492 1
RUN BLOCK 1 21 492 2
RUN BLOCK 3 21 492 2
RUN BLOCK 1 22 492 1
RUN BLOCK 3 22 492 1
RUN BLOCK 1 23 492 2
RUN BLOCK 3 23 492 2
RUN BLOCK 1 24 492 1
RUN BLOCK 3 24 492 1
RUN BLOCK 1 25 492 2
RUN BLOCK 3 25 492 2
RUN BLOCK 1 26 492 1
RUN BLOCK 3 26 492 1
RUN BLOCK 1 27 492 2
RUN BLOCK 3 27 492 2
RUN BLOCK 1 28 492 1
RUN BLOCK 3 28 492 1
RUN BLOCK 1 29 492 2
RUN BLOCK 3 29 492 2
RUN BLOCK 1 30 492 1
RUN BLOCK 3 30 492 1
RUN BLOCK 1 31 492 2
RUN BLOCK 3 31 492 2
RUN BLOCK 1 32 492 1
RUN BLOCK 3 32 492 1
RUN BLOCK 1 33 492 2
RUN BLOCK 3 33 492 2
RUN BLOCK 1 34 492 1
RUN BLOCK 3 34 492 1
RUN BLOCK 1 35 492 2
RUN BLOCK 3 35 492 2
RUN BLOCK 1 36 492 1
RUN BLOCK 3 36 492 1
RUN BLOCK 1 37 492 2
RUN BLOCK 3 37 492 2
RUN BLOCK 1 38 492 1
RUN BLOCK 3 38 492 1
RUN BLOCK 1 39 492 2
RUN BLOCK 3 39 492 2
RUN BLOCK 1 40 492 1
RUN BLOCK 3 40 492 1
RUN BLOCK 1 41 492 2
RUN BLOCK 3 41 492 2
RUN BLOCK 1 42 492 1
RUN BLOCK 3 42 492 1
RUN BLOCK 1 43 492 2
RUN BLOCK 3 43 492 2
RUN BLOCK 1 4

RUN BLOCK 3 0 509 2
RUN BLOCK 1 0 510 1
RUN BLOCK 2 0 510 1
RUN BLOCK 3 0 510 1
RUN BLOCK 1 1 510 2
RUN BLOCK 3 1 510 2
RUN BLOCK 1 2 510 1
RUN BLOCK 3 2 510 1
RUN BLOCK 1 3 510 2
RUN BLOCK 3 3 510 2
RUN BLOCK 1 4 510 1
RUN BLOCK 3 4 510 1
RUN BLOCK 1 5 510 2
RUN BLOCK 3 5 510 2
RUN BLOCK 1 6 510 1
RUN BLOCK 3 6 510 1
RUN BLOCK 1 7 510 2
RUN BLOCK 3 7 510 2
RUN BLOCK 1 8 510 1
RUN BLOCK 3 8 510 1
RUN BLOCK 1 9 510 2
RUN BLOCK 3 9 510 2
RUN BLOCK 1 10 510 1
RUN BLOCK 3 10 510 1
RUN BLOCK 1 11 510 2
RUN BLOCK 3 11 510 2
RUN BLOCK 1 12 510 1
RUN BLOCK 3 12 510 1
RUN BLOCK 1 13 510 2
RUN BLOCK 3 13 510 2
RUN BLOCK 1 14 510 1
RUN BLOCK 3 14 510 1
RUN BLOCK 1 15 510 2
RUN BLOCK 3 15 510 2
RUN BLOCK 1 16 510 1
RUN BLOCK 3 16 510 1
RUN BLOCK 1 17 510 2
RUN BLOCK 3 17 510 2
RUN BLOCK 1 18 510 1
RUN BLOCK 3 18 510 1
RUN BLOCK 1 19 510 2
RUN BLOCK 3 19 510 2
RUN BLOCK 1 20 510 1
RUN BLOCK 3 20 510 1
RUN BLOCK 1 21 510 2
RUN BLOCK 3 21 510 2
RUN BLOCK 1 22 510 1
RUN BLOCK 3 22 510 1
RUN BLOCK 1 23

RUN BLOCK 3 28 526 1
RUN BLOCK 1 29 526 2
RUN BLOCK 3 29 526 2
RUN BLOCK 1 30 526 1
RUN BLOCK 3 30 526 1
RUN BLOCK 1 31 526 2
RUN BLOCK 3 31 526 2
RUN BLOCK 1 32 526 1
RUN BLOCK 3 32 526 1
RUN BLOCK 1 33 526 2
RUN BLOCK 3 33 526 2
RUN BLOCK 1 34 526 1
RUN BLOCK 3 34 526 1
RUN BLOCK 1 35 526 2
RUN BLOCK 3 35 526 2
RUN BLOCK 1 36 526 1
RUN BLOCK 3 36 526 1
RUN BLOCK 1 37 526 2
RUN BLOCK 3 37 526 2
RUN BLOCK 1 38 526 1
RUN BLOCK 3 38 526 1
RUN BLOCK 1 39 526 2
RUN BLOCK 3 39 526 2
RUN BLOCK 1 40 526 1
RUN BLOCK 3 40 526 1
RUN BLOCK 1 41 526 2
RUN BLOCK 3 41 526 2
RUN BLOCK 1 42 526 1
RUN BLOCK 3 42 526 1
RUN BLOCK 1 43 526 2
RUN BLOCK 3 43 526 2
RUN BLOCK 1 44 526 1
RUN BLOCK 3 44 526 1
RUN BLOCK 1 45 526 2
RUN BLOCK 3 45 526 2
RUN BLOCK 1 46 526 1
RUN BLOCK 3 46 526 1
RUN BLOCK 1 47 526 2
RUN BLOCK 3 47 526 2
RUN BLOCK 1 48 526 1
RUN BLOCK 3 48 526 1
RUN BLOCK 1 0 527 2
RUN BLOCK 2 0 527 2
RUN BLOCK 3 0 527 2
RUN BLOCK 1 0 528 1
RUN BLOCK 2 0 528 1
RUN BLOCK 3 0 528 1
RUN BLOCK 1 1 528 2

RUN BLOCK 3 6 544 1
RUN BLOCK 1 7 544 2
RUN BLOCK 3 7 544 2
RUN BLOCK 1 8 544 1
RUN BLOCK 3 8 544 1
RUN BLOCK 1 9 544 2
RUN BLOCK 3 9 544 2
RUN BLOCK 1 10 544 1
RUN BLOCK 3 10 544 1
RUN BLOCK 1 11 544 2
RUN BLOCK 3 11 544 2
RUN BLOCK 1 12 544 1
RUN BLOCK 3 12 544 1
RUN BLOCK 1 13 544 2
RUN BLOCK 3 13 544 2
RUN BLOCK 1 14 544 1
RUN BLOCK 3 14 544 1
RUN BLOCK 1 15 544 2
RUN BLOCK 3 15 544 2
RUN BLOCK 1 16 544 1
RUN BLOCK 3 16 544 1
RUN BLOCK 1 17 544 2
RUN BLOCK 3 17 544 2
RUN BLOCK 1 18 544 1
RUN BLOCK 3 18 544 1
RUN BLOCK 1 19 544 2
RUN BLOCK 3 19 544 2
RUN BLOCK 1 20 544 1
RUN BLOCK 3 20 544 1
RUN BLOCK 1 21 544 2
RUN BLOCK 3 21 544 2
RUN BLOCK 1 22 544 1
RUN BLOCK 3 22 544 1
RUN BLOCK 1 23 544 2
RUN BLOCK 3 23 544 2
RUN BLOCK 1 24 544 1
RUN BLOCK 3 24 544 1
RUN BLOCK 1 25 544 2
RUN BLOCK 3 25 544 2
RUN BLOCK 1 26 544 1
RUN BLOCK 3 26 544 1
RUN BLOCK 1 27 544 2
RUN BLOCK 3 27 544 2
RUN BLOCK 1 28 544 1
RUN BLOCK 3 28 544 1
RUN BLOCK 1 29 544 2
RUN BLOCK 3 29 544 2
RUN BLOCK 1 30 544 1

In [ ]:
#                   !!! END OF DEBUGGING PROCESS !!!

In [ ]:
#                          !!! ВОТ И САМИ РАСЧЕТЫ !!!
class CalculateLosses():
   
    def __init__(self, podstan, prisoed=None, harm=None, t_izmer=None, distance=None, step=None):
        
        # Проверка на правильность введенных данных для расчета
        assert type(podstan) == PodStans, "Podstan must be PodStand object"
        assert type(prisoed) == int and prisoed != 0 and prisoed <= len(podstan.vse_prisoed), f"Prisoed number must be integer and not equal to zero. And within the range 1 and {len(podstan.vse_prisoed)}"
        if type(harm) == str:
            assert harm.upper() == "ALL", "Type 'ALL' to calculate all harmonics"
            self.all_harms = True
        else:
            assert type(harm) == int and harm < 50 and harm > 1, "Harmonic number must be integer and betweet values 2 and 49"
            self.all_harms = False
        assert type(distance) == int and distance != 0, "Distance number must be integer and not equal to zero"
        assert type(step) in [int, float] and step != 0, "Step size must be integer or float and not equal to zero"

        # Работа с определенным присоединением
        prisoed = getattr(podstan, "prisoed_"+str(prisoed))

        # Проверка налчичя данного измерения по времени в файле EXCEL
        self.num_record = len(prisoed.faza_B("U").index)
        assert t_izmer <= self.num_record and t_izmer >= 1, f"Time ticks must be within the range 1 and {self.num_record-1}"

        self.podstan = podstan
        self.prisoed = prisoed
        self.harm = harm
        self.t_izmer = t_izmer
        self.distance = distance
        self.step = step
        self.tros = 0
        
        self.result_dict = {
            "U_n": [],
            "U_k": [],
            "I_n": [],
            "I_k": [],
        }
        
        
    def prepare_columnsrows(self):
    # Составление векторов-столбцов тока и напряжения для начала линии второй гармоники 
        for i in range(self.t_izmer):
            if i == 0:
                volt_phaseA_har_n = self.prisoed.faza_A.get_harmonic(self.harm, 'U')[i]
                volt_phaseB_har_n = self.prisoed.faza_B.get_harmonic(self.harm, 'U')[i]
                volt_phaseC_har_n = self.prisoed.faza_C.get_harmonic(self.harm, 'U')[i]
                curr_phaseA_har_n = self.prisoed.faza_A.get_harmonic(self.harm, 'I')[i]
                curr_phaseB_har_n = self.prisoed.faza_B.get_harmonic(self.harm, 'I')[i]
                curr_phaseC_har_n = self.prisoed.faza_C.get_harmonic(self.harm, 'I')[i]

                U_n = np.array([[volt_phaseA_har_n], [volt_phaseB_har_n], [volt_phaseC_har_n], [0]])
                I_n = np.array([[curr_phaseA_har_n], [curr_phaseB_har_n], [curr_phaseC_har_n], [0]])
           

            volt_phaseA_har_k = self.prisoed.faza_A.get_harmonic(self.harm, 'U')[i]
            volt_phaseB_har_k = self.prisoed.faza_B.get_harmonic(self.harm, 'U')[i]
            volt_phaseC_har_k = self.prisoed.faza_C.get_harmonic(self.harm, 'U')[i]
            curr_phaseA_har_k = self.prisoed.faza_A.get_harmonic(self.harm, 'I')[i]
            curr_phaseB_har_k = self.prisoed.faza_B.get_harmonic(self.harm, 'I')[i]
            curr_phaseC_har_k = self.prisoed.faza_C.get_harmonic(self.harm, 'I')[i]

            U_k = np.array([[volt_phaseA_har_k], [volt_phaseB_har_k], [volt_phaseC_har_k], [0]])
            I_k = np.array([[curr_phaseA_har_k], [curr_phaseB_har_k], [curr_phaseC_har_k], [0]])

        
            # Нахождение векторов-столбцов постоянных интергирования
            E1 = np.identity(4)
            E2 = np.identity(4)
            E3 = np.identity(4)
            E4 = np.identity(4)

            pos_U_exponent, neg_U_exponent = rashet.calculate_exponentials(lambda_U, self.distance)
            pos_I_exponent, neg_I_exponent = rashet.calculate_exponentials(lambda_I, self.distance)

            GU = np.array([[[E1], [E2]],[[neg_U_exponent[self.harm-1]], [pos_U_exponent[self.harm-1]]]])
            GI = np.array([[[E3], [E4]],[[neg_I_exponent[self.harm-1]], [pos_I_exponent[self.harm-1]]]])

            GU_inv = inv(GU)
            GI_inv = inv(GI)

            self.Aw = np.dot(GU_inv[0][0][0], U_n).reshape(4,1) + np.dot(GU_inv[0][1][0], U_k).reshape(4,1)
            self.Bw = np.dot(GU_inv[1][0][0], U_n).reshape(4,1) + np.dot(GU_inv[1][1][0], U_k).reshape(4,1)
            self.Cw = np.dot(GI_inv[0][0][0], I_n).reshape(4,1) + np.dot(GI_inv[0][1][0], I_k).reshape(4,1)
            self.Dw = np.dot(GI_inv[0][0][0], I_n).reshape(4,1) + np.dot(GI_inv[0][1][0], I_k).reshape(4,1)
            
            
            Yw = inv(Y[self.harm-1])*-1
            Zw = inv(Z[self.harm-1])*-1

            lu = lambda_U[self.harm-1]
            li = lambda_I[self.harm-1]

            res_U_n = np.dot(Yw, (np.dot(-1*li, self.Cw) + np.dot(li, self.Dw)))
            res_I_n = np.dot(Zw, (np.dot(-1*lu, self.Aw) + np.dot(lu, self.Bw)))

            self.result_dict["U_n"].append(res_U_n)
            self.result_dict["I_n"].append(res_I_n)
            
            temp_I_holder = []
            temp_U_holder = []
            
            for step in range(1000, self.distance+1, 1000):
                pos_U_exponent, neg_U_exponent = rashet.calculate_exponentials(lambda_U, step)
                pos_I_exponent, neg_I_exponent = rashet.calculate_exponentials(lambda_I, step)
                
                res_U_k = np.dot(Yw, (np.dot(np.dot(-1*li, neg_I_exponent[self.harm-1]), self.Cw) + np.dot(np.dot(li, pos_I_exponent[self.harm-1]), self.Dw)))
                res_I_k = np.dot(Zw, (np.dot(np.dot(-1*lu, neg_U_exponent[self.harm-1]), self.Aw) + np.dot(np.dot(lu, pos_U_exponent[self.harm-1]), self.Bw)))
                
                temp_U_holder.append(res_U_k)
                temp_I_holder.append(res_I_k)
            
            self.result_dict["U_k"].append(temp_U_holder)
            self.result_dict["I_k"].append(temp_I_holder)               

        return self.result_dict    
            
    
    def calculate_losses(self, Is, phase):
        
        assert type(phase) == str, "Phase type must be defined as a string!"
        if phase.lower().strip() == "a":
            iphase = 0
        elif phase.lower().strip() == "b":
            iphase = 1
        elif phase.lower().strip() == "c":
            iphase = 2
        elif phase.lower().strip() == "t":
            iphase = 3
    
        phaseX = [(phases[iphase][0]) for phases in Is]; phase2sum = sum(list(map(lambda x:np.power(x,2), phaseX)))
        r = pogon_aktiv_soprotiv_faz[iphase][self.harm-1]
        overall_losses = r * phase2sum
        
        return overall_losses   


In [ ]:
if __name__ == "__main__":
#     start = time.time()
    calcs = CalculateLosses(PodS_zelenaya, prisoed=1, harm=2, t_izmer=560, distance=5000, step=1000)
    result_dict = calcs.prepare_columnsrows()
#     Losses = calcs.calculate_losses(result_dict["I_k"], "A")
#     end = time.time()
#     print("Execution time(sec): ", end - start)

In [ ]:
phase = 0 # Phase A
harm = 2 # Harmonic 2

all_vals = []
for time in range(560):
    temp_vals = []
    for step in range(5):
        temp_vals.append((result_dict["I_k"][time][step][phase].real**2) * pogon_aktiv_soprotiv_faz[phase][harm-1])
    
    all_vals.append(sum(temp_vals))
 

print(sum(all_vals))
for item in all_vals:
    print(item)

In [ ]:
for i, j in pairwise_1([1,2,3,4,5,6,7,8,9]):
    print(i,j)